In [36]:
import requests
import os
import time

In [62]:
chambers = [
    "senate",
    "house"
]
congress_numbers = [117, 118, 119] # this means we can do at least 3 cycles xd
currentest_congress = 0

In [132]:
member_data = []
for chamber in chambers:
    for congress_number in congress_numbers:
        time.sleep(3)
        response = requests.get(f"https://api.propublica.org/congress/v1/{congress_number}/{chamber}/members.json", headers=headers_dict)
        if response.ok and response.json()["status"] != 'ERROR':
            members = response.json()["results"][0]["members"]
            if len(members) > 0:
                currentest_congress = congress_number if congress_number > currentest_congress else currentest_congress
            for member in members:
                member_data.append({"name": f"{member['first_name']} {member['last_name']}", "chamber": chamber, "congress_no": congress_number, "id":member["id"]})
        else: 
            print("idk if this year exists lmaooo")

In [74]:
def assign_opensecrets_data(primary_subject):
    return {
         "Health":                                          ["H", "Health", "Health"],
         "Government Operations and Politics":              ["Z", "Joint Candidate Cmtes", "Joint Candidate Cmtes"],
         "International Affairs":                           ["Q", "Ideology/Single-Issue", "Ideological/Single-Issue"],
         "Congress":                                        ["Z", "Joint Candidate Cmtes", "Joint Candidate Cmtes"],
         "Crime and Law Enforcement":                       ["P", "Labor", "Labor"],
         "Taxation":                                        ["Q", "Ideology/Single-Issue", "Ideological/Single-Issue"],
         "Armed Forces and National Security":              ["D", "Defense", "Defense"],
         "Public Lands and Natural Resources":              ["E", "Energy/Nat Resource", "Energy & Natural Resources"],
         "Education":                                       ["W", "Other", "Other"],
         "Transportation and Public Works":                 ["M", "Transportation", "Transportation"],
         "Immigration":                                     ["Q", "Ideology/Single-Issue", "Ideological/Single-Issue"],
         "Science, Technology, Communications":             ["C", "Communic/Electronics", "Communications/Electronics"],
         "Labor and Employment":                            ["P", "Labor", "Labor"],
         "Commerce":                                        ["N", "Misc Business", "Misc Business"],
         "Environmental Protection":                        ["Q", "Ideology/Single-Issue", "Ideological/Single-Issue"],
         "Finance and Financial Sector":                    ["F", "Finance/Insur/RealEst", "Finance, Insurance & Real Estate"],
         "Energy":                                          ["E", "Energy/Nat Resource", "Energy & Natural Resources"],
         "Civil Rights and Liberties, Minority Issues":     ["Q", "Ideology/Single-Issue", "Ideological/Single-Issue"],
         "Agriculture and Food":                            ["A", "Agribusiness", "Agribusiness"],
         "Native Americans":                                ["Q", "Ideology/Single-Issue", "Ideological/Single-Issue"],
         "Economics and Public Finance":                    ["F", "Finance/Insur/RealEst", "Finance, Insurance & Real Estate"],
         "Law":                                             ["K", "Lawyers & Lobbyists", "Lawyers & Lobbyists"],
         "Housing and Community Development":               ["C", "Construction", "Construction"],
         "Emergency Management":                            ["W", "Other", "Other"],
         "Social Welfare":                                  ["P", "Labor", "Labor"],
         "Sports and Recreation":                           ["N", "Misc Business", "Misc Business"],
         "Foreign Trade and International Finance":         ["N", "Misc Business", "Misc Business"],
         "Families":                                        ["Y", "Unknown", "Unknown"],
         "Arts, Culture, Religion":                         ["W", "Other", "Other"],
         "Water Resources Development":                     ["E", "Energy/Nat Resource", "Energy & Natural Resources"],
         "Animals":                                         ["A", "Agribusiness", "Agribusiness"]
    }.get(primary_subject, ["Z", "PROBLEM", "PROBLEM"])
     

In [135]:
def find_vote_information(bill_slug, congress_number):
    response = requests.get(f'https://api.propublica.org/congress/v1/{congress_number}/bills/{bill_slug}.json', headers=headers_dict)
    if response.ok and response.json()["status"] == "OK":
        if len(response.json()["results"][0]["votes"]) > 0:
            vote_url = response.json()["results"][0]["votes"][0]["api_url"]
            return parse_voter_data(vote_url)
    return None

def parse_voter_data(vote_url):
    response = requests.get(vote_url, headers=headers_dict)
    if response.ok and response.json()["status"] == "OK":
        vote_dict = {}
        for position in response.json()["results"]["votes"]["vote"]["positions"]:
            vote_dict[position["member_id"]] = position["vote_position"]
        return {"vote_dict":vote_dict, "raw_response": response.json()["results"]["votes"]["vote"], "vote_uri": vote_url}
    return None

In [141]:
vote_data = []
response = requests.get('https://api.propublica.org/congress/v1/117/house/bills/active.json?offset=0', headers=headers_dict)
if response.ok and response.json()["status"] == "OK":
    bills = response.json()["results"][0]["bills"]
    for bill in bills:
        bill_data = {}
        vote_information = find_vote_information(bill["bill_slug"], currentest_congress)
        bill_data["vote_information"] = vote_information
        if vote_information:
            bill_data["vote_date"] = vote_information["raw_response"]["date"]
            bill_data['bill_slug'] = bill['bill_slug']
            bill_data['bill_id'] = bill['bill_id']
            bill_data['bill_number'] = vote_information['raw_response']['bill']['number']
            bill_data['sponsor_id'] = bill["sponsor_id"]
            bill_data['sponsor_uri'] = bill['sponsor_uri']
            bill_data['bill_uri'] = bill['bill_uri']
            bill_data['title'] = bill['title']
            bill_data['latest_action'] = vote_information['raw_response']['bill']['latest_action']
            bill_data['short_title'] = bill['short_title']
            bill_data['primary_subject'] = bill['primary_subject']
            bill_data["opensecrets_sector_data"] = assign_opensecrets_data(bill["primary_subject"])
            vote_data.append(bill_data)

In [143]:
vote_data[0]

{'votre_information': {'vote_dict': {'A000370': 'Yes',
   'A000055': 'Yes',
   'A000371': 'Yes',
   'A000372': 'Not Voting',
   'A000376': 'Yes',
   'A000369': 'Yes',
   'A000377': 'Yes',
   'A000375': 'Yes',
   'A000148': 'Yes',
   'A000378': 'Yes',
   'B001291': 'Yes',
   'B001298': 'Yes',
   'B001307': 'Yes',
   'B001306': 'Yes',
   'B001299': 'Yes',
   'B001282': 'Yes',
   'B001300': 'Yes',
   'B001270': 'Yes',
   'B001281': 'Yes',
   'B000668': 'Yes',
   'B001287': 'Yes',
   'B001301': 'Yes',
   'B001292': 'Yes',
   'B000740': 'Yes',
   'B001302': 'Yes',
   'B001257': 'Yes',
   'B000490': 'Yes',
   'B001311': 'Yes',
   'B000574': 'Yes',
   'B001303': 'Yes',
   'B000825': 'Yes',
   'B001278': 'Yes',
   'B001295': 'Yes',
   'B001312': 'Yes',
   'B001223': 'Yes',
   'B001296': 'Yes',
   'B000755': 'Yes',
   'B001274': 'Yes',
   'B001304': 'Yes',
   'B001313': 'Yes',
   'B001285': 'Yes',
   'B001260': 'Yes',
   'B001297': 'Yes',
   'B001275': 'Yes',
   'B001305': 'Yes',
   'B001309': 